Import Libraries

In [ ]:
import scipy as sp
from scipy import optimize
from scipy.optimize import fsolve
from scipy import interpolate

# Import pandas
import pandas as pd

# Import numpy
import numpy as np
from numpy import * 
from numpy.linalg import multi_dot
import numpy_financial as npf

# Plot settings
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = 16, 8

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# IBT or Pension or Retirement
PLAN = 'IBT'

UPS_Contr_Pctg = 0

In [ ]:
df_CF = pd.read_excel("2021_PBO_CF.xlsx",sheet_name = "PBO Cashflows Summary", skiprows = [0,1,2], usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_CF

In [ ]:
df_CF = df_CF.divide(12)
df_CF

In [ ]:
df=df_CF.copy()

#set start date and end date
start_date = df.index.min() - pd.DateOffset(day=0)
end_date = df.index.max() + pd.DateOffset(day=31)

#create new dataframe monthly index
dates = pd.date_range(start_date, end_date, freq='M')
dates.name = 'date'

#reindex yearly dataframe to monthly dataframe
df = df.reindex(dates, method='ffill')

#export monthly dataframe to excel
#filepath = 'monthly_data.xlsx'
#writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
#df.to_excel(writer, sheet_name='data')
#writer.save()
df_CF = df
df_CF

In [ ]:
df_CF_add = pd.read_excel("2021 Liability Cash Flows.xlsx",sheet_name = "CF", usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_CF_add

In [ ]:
df_CF  = df_CF_add.append(df_CF)
df_CF

In [ ]:
from itertools import count, takewhile
def frange(start, stop, step):
        return takewhile(lambda x: x< stop, count(start, step))
t = list(frange((1/12), 79.41666666666, (1/12)))

In [ ]:
df_t=pd.DataFrame(t)
df_t

In [ ]:
df_CF['Time'] = df_t.values
df_CF

In [ ]:
#IBT = df_CF[PLAN]
#RP = df_CF[PLAN]
CF = np.array(df_CF[PLAN])
Time_CF = df_CF.index
DF = np.array(df_CF['Time'])
CF

In [ ]:
# from PyCurve.curve import Curve

In [ ]:
df_Raw_FTSE_YC = pd.read_excel("Raw FTSE Data.xlsx",skiprows = [0,2],usecols = [*range(0, 145)], na_values=[""])
#dataset.set_index('Date')
Raw_FTSE_YC = df_Raw_FTSE_YC.copy()
#Raw_FTSE_YC = pd.DataFrame(Raw_FTSE_YC)
#df_Raw_FTSE_YC.set_index('Date')
col = Raw_FTSE_YC.columns
Raw_FTSE_YC


In [ ]:
raw_liab_ibt_dict={}
tr = Raw_FTSE_YC['Date']
# list of raw (not interpolated) times to maturity
yr = Raw_FTSE_YC[col] 
# list of raw (not interpolated) yields
t = list(frange(0.5, 30.08, (1/12))) # interpolating in range 1..30 years

for col in Raw_FTSE_YC.columns:
    #print('col: {}'.format(col))
    y = []
    interp = sp.interpolate.interp1d(tr, Raw_FTSE_YC[col] , bounds_error=False, fill_value=sp.nan)
#for col in Raw_FTSE_YC.columns:
    #for col in FTSE_YC.columns:
    for i in t:
            value = float(interp(i))
            if not sp.isnan(value): # Don't include out-of-range values
                y.append(value)
                End_Rate = [y[-1]] * 592
                Beg_Rate = [y[0]] * 5
            raw_liab_ibt_dict[col] = Beg_Rate + y + End_Rate
raw_liab_ibt_dict[col]
#curve = Curve(t,y)
#print("curve.get_rate:{},curve.get_time:{}".format(curve.get_rate,curve.get_time))
#print(curve.get_rate)
#print(curve.get_time)

In [ ]:
int_df = pd.DataFrame(raw_liab_ibt_dict)
int_df = int_df.drop(['Date'], axis=1)
int_df = int_df.iloc[:, ::-1]
int_df

In [ ]:
liab_plan_dict={}
for col in int_df.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF[j]/((1+int_df[col][j]/100)**DF[j]))
    liab_plan_dict[col] = temp_pv
len(liab_plan_dict)

In [ ]:
pv_df = pd.DataFrame(liab_plan_dict, index = ['Present Values']).transpose()
col = pv_df.columns
np.negative(pv_df['Present Values'][0])

In [ ]:
Plan_Return_YC = np.zeros(len(liab_plan_dict)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC[i] = ((pv_df['Present Values'][i+1])/pv_df['Present Values'][i])-1
Plan_Return_YC

In [ ]:
Plan_Returns_YC = pd.DataFrame(Plan_Return_YC)
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability Returns & PV.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC.to_excel(writer, sheet_name='returns')
pv_df.to_excel(writer, sheet_name='pv')
writer.save()

In [ ]:
df_SC = pd.read_excel("2021_PBO_CF.xlsx",sheet_name = "Service Cost Cashflows Summary", skiprows = [0,1,2], usecols=[0,1,2,3], na_values=[""],index_col=0)
#dataset.set_index('Year')?
df_SC

In [ ]:
df_SC = df_SC.divide(12)
df_SC

In [ ]:
df=df_SC.copy()

#set start date and end date
start_date = df.index.min() - pd.DateOffset(day=0)
end_date = df.index.max() + pd.DateOffset(day=31)

#create new dataframe monthly index
dates = pd.date_range(start_date, end_date, freq='M')
dates.name = 'date'

#reindex yearly dataframe to monthly dataframe
df = df.reindex(dates, method='ffill')

#export monthly dataframe to excel
#filepath = 'monthly_data.xlsx'
#writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
#df.to_excel(writer, sheet_name='data')
#writer.save()
df_SC = df
df_SC

In [ ]:
df_SC_add = pd.read_excel("2021 Liability Cash Flows.xlsx",sheet_name = "SC", usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_SC_add

In [ ]:
df_SC  = df_SC_add.append(df_SC)
df_SC

In [ ]:
col_SC = df_SC.columns
col_CF = df_CF.columns
df_CF_Tot = UPS_Contr_Pctg*df_SC[col_SC] + df_CF[col_CF]
df_CF_Tot

In [ ]:
df_CF_Tot['Time'] = df_t.values
df_CF_Tot

In [ ]:
CF_Tot = df_CF_Tot[PLAN]
Time_CF_Tot = df_CF_Tot.index
DF_Tot=df_CF_Tot['Time']

In [ ]:
liab_plan_dict_tot={}
for col in int_df.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF_Tot[j]/((1+int_df[col][j]/100)**DF_Tot[j]))
    liab_plan_dict_tot[col] = temp_pv
liab_plan_dict_tot


In [ ]:
pv_df_tot = pd.DataFrame(liab_plan_dict_tot, index = ['Present Values']).transpose()

pv_df_tot

In [ ]:
Plan_Return_YC_Tot = np.zeros(len(liab_plan_dict_tot)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict_tot)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC_Tot[i] = ((pv_df_tot['Present Values'][i+1])/pv_df_tot['Present Values'][i])-1
Plan_Return_YC_Tot

In [ ]:
Plan_Returns_YC_Tot = pd.DataFrame(Plan_Return_YC_Tot)
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability & SC Returns & PV.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC_Tot.to_excel(writer, sheet_name='returns')
pv_df_tot.to_excel(writer, sheet_name='pv')
writer.save()

In [ ]:
Plan_DR = pd.read_excel("UPS Pension - Historical Liability Data - 9.30.21.xlsx",sheet_name = PLAN ,skiprows = [0,1,2,3],usecols=[1,2], na_values=[""],index_col=0)
#dataset.set_index('Year')
len(Plan_DR)


In [ ]:
col_DR = Plan_DR.columns
#df_DR_M =  (1+Plan_DR)**(1/12)-1
df_DR_M = Plan_DR
IRR = df_DR_M['IRR']
IRR

In [ ]:
Plan_PV_DR = np.zeros(len(df_DR_M))
for j in range (len(df_DR_M)):
    for i in range (len(Time_CF)):
        Plan_PV_DR[j] += (CF[i]/((1+IRR[j])**DF[i]))
Plan_PV_DR

In [ ]:
pv_df_dr = pd.DataFrame(Plan_PV_DR, columns=['Present Values'])

pv_df_dr

In [ ]:
pv_df_dr = pd.DataFrame(Plan_PV_DR, columns=['Present Values'])
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability PV Using DR.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
pv_df_dr.to_excel(writer, sheet_name='Present Values')
writer.save()

In [ ]:
Initial_CF = np.negative(pv_df_tot['Present Values'])
type(Initial_CF)


In [ ]:
CF

In [ ]:
DF


In [ ]:
def npv(irr, cfs, yrs):  
    return np.sum(cfs / (1. + irr) ** yrs)

def irr(cfs, yrs, x0, **kwargs):
    return np.asscalar(fsolve(npv, x0=x0, args=(cfs, yrs), **kwargs))


# NPV = np.zeros(len(df_DR_M))
# for j in range (len(df_DR_M)):
#     for i in range (len(Time_CF)):
#         NPV[j] += npv(IRR[j], CF[i], DF[i])
# NPV 

IRR = np.zeros(len(df_DR_M))
for j in range (len(df_DR_M)):
    cfs = np.append(np.negative(pv_df_dr['Present Values'][j]),CF)
    yrs = np.append(0, DF)
    IRR[j] += irr(cfs, yrs, .03)
IRR 

